In [ ]:
import duckdb
from pathlib import Path
# pyarrow y pyarrow.parquet se importan implícitamente por DuckDB para operaciones con Parquet,
# pero no se usan directamente en este script. Se pueden omitir si no hay otras dependencias.

# Definición de rutas utilizando pathlib para una mejor gestión de rutas
RUTA_BRONZE_DETALLISTAS = Path('../data/bronze_detallistas.parquet')
RUTA_BRONZE_VENTAS = Path('../data/bronze_ventas.parquet')
RUTA_SILVER_VENTAS = Path('../data/silver_ventas_establecimiento.parquet')

def crear_parquet_ventas_silver(ruta_salida: Path):
    """
    Crea una tabla silver de ventas agregadas por establecimiento y otras dimensiones.

    Lee datos de ventas y detallistas desde archivos Parquet de la capa bronze,
    los une, agrega las ventas y guarda el resultado en un nuevo archivo Parquet
    en la capa silver.

    Args:
        ruta_salida: Objeto Path que indica dónde guardar el archivo Parquet resultante.
    """
    # Se utiliza un gestor de contexto para asegurar que la conexión se cierre automáticamente
    with duckdb.connect() as con:
        # Carga las tablas bronze en DuckDB
        # Se usa CREATE OR REPLACE para asegurar que la tabla se cree o reemplace si ya existe
        con.sql(f"CREATE OR REPLACE TABLE detallistas AS SELECT * FROM read_parquet('{RUTA_BRONZE_DETALLISTAS}')")
        con.sql(f"CREATE OR REPLACE TABLE ventas AS SELECT * FROM read_parquet('{RUTA_BRONZE_VENTAS}')")

        # Consulta SQL para unir, agregar y transformar los datos
        consulta_transformacion = f"""
        COPY (
            SELECT
                d.establecimiento,
                v.material,
                v.calday,
                v.promo_id,
                SUM(v.volume_ap) AS volume_ap,
                SUM(v.cantidad_umb) AS cantidad_umb,
                d.type AS tipo
            FROM ventas v
            JOIN detallistas d
            ON CAST(v.detallista AS VARCHAR) = d.detallista  -- Asegura la unión correcta de tipos
            GROUP BY d.establecimiento, v.material, v.calday, v.promo_id, d.type
        ) TO '{ruta_salida}' (FORMAT PARQUET)
        """
        
        con.execute(consulta_transformacion)
        print(f"Archivo Parquet silver creado exitosamente en: {ruta_salida}")

def mostrar_muestra_datos_silver(ruta_archivo_silver: Path, numero_filas: int = 5):
    """
    Muestra las primeras N filas del archivo Parquet silver especificado.

    Args:
        ruta_archivo_silver: Objeto Path del archivo Parquet a leer.
        numero_filas: Número de filas a mostrar.
    """
    with duckdb.connect() as con:
        print(f"\nPrimeras {numero_filas} filas de {ruta_archivo_silver.name}:")
        con.sql(f"SELECT * FROM read_parquet('{ruta_archivo_silver}') LIMIT {numero_filas}").show()

# --- Bloque principal de ejecución del script ---
if __name__ == "__main__":
    # Crear el archivo Parquet en la capa silver
    crear_parquet_ventas_silver(RUTA_SILVER_VENTAS)

    # Mostrar una muestra de los datos generados en la tabla silver
    mostrar_muestra_datos_silver(RUTA_SILVER_VENTAS)



First 5 rows of silver_ventas.parquet:
┌─────────────────┬──────────┬─────────────────────┬──────────────────────┬───────────┬──────────────┬─────────────────┐
│ establecimiento │ material │       calday        │       promo_id       │ volume_ap │ cantidad_umb │      tipo       │
│     varchar     │ varchar  │      timestamp      │       varchar        │  double   │    double    │     varchar     │
├─────────────────┼──────────┼─────────────────────┼──────────────────────┼───────────┼──────────────┼─────────────────┤
│ 8100240876      │ TB8      │ 2024-11-26 00:00:00 │ NULL                 │       8.0 │          1.0 │ Bar Cervecería  │
│ 8100032055      │ PI13     │ 2024-11-26 00:00:00 │ NULL                 │      7.92 │          1.0 │ Restaurante     │
│ 8100258434      │ FL13SPN  │ 2024-11-26 00:00:00 │ NULL                 │      23.1 │          2.0 │ Bar Cervecería  │
│ 8100036860      │ VO13     │ 2024-11-26 00:00:00 │ 00000000000080619348 │       0.0 │          0.0 │ Bar Cervec